In [1]:
import molsysmt as msm
import numpy as np
from molsysmt import pyunitwizard as puw

In [2]:
molsys_AF = msm.convert('AF-P52789-F1')

In [3]:
molsys_AF = msm.build.add_missing_hydrogens(molsys_AF, pH=7.4)

In [4]:
msm.build.remove_bonds(molsys_AF)

In [5]:
x = msm.build.get_missing_bonds(molsys_AF)

In [6]:
msm.view(molsys_AF)

NGLWidget()